In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [11]:
STORM_NUMBER = 16
TITLE = "4 February 2021"
SUBTITLE = "8 AM 5 February 2021"
sts = utc(2021, 2, 4, 12)
ets = utc(2021, 12, 30, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [9]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [41]:
cull = ['DVNI4', 'OSAI4', 1003, 989, 'IWAI4', 'GNDI4', 1041, 686, 772, 577, 582, 371]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    [907, 2],
    ['AESI4', 2.5],
    [1042, 2],
    [1041, 2.5],
    [371, 3],
    [581, 2.5],
    ['IA-PK-21', 2.5],
    ['CINI4', 2],
    [903, 2],
    ['SLBI4', 2],
    [471, 2.5],
    [621, 2.5],
    [653, 2.5],
    ['PERI4', 2.5],
    
    
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=6, upper=10)
res = mp.postprocess(filename='210205.png')
mp.close()

     state  wfo  val       lon      lat                              geo  \
20      MI  APX  9.6  -84.3700  46.4800   POINT (1187451.979 280552.196)   
21      WI  GRB  9.5  -89.8800  46.1300    POINT (777327.290 174556.867)   
22      WI  GRB  9.5  -89.1100  44.5600      POINT (859567.275 8899.259)   
23      WY  CYS  9.0 -106.7100  41.0600  POINT (-562150.128 -415119.858)   
24      WI  GRB  9.0  -87.9400  45.3600    POINT (937897.775 110195.872)   
...    ...  ...  ...       ...      ...                              ...   
1171    IA  DMX  6.0  -93.7233  41.7361   POINT (520395.505 -342991.551)   
1172    WI  ARX  6.0  -91.2295  43.5705   POINT (704938.954 -120961.697)   
1173    MI  APX  6.0  -85.2672  45.3172   POINT (1143996.831 139995.056)   
1174    CO  BOU  6.0 -105.7080  39.6552  POINT (-488708.876 -577273.448)   
1175    CO  PUB  6.0 -106.3513  39.0935  POINT (-548187.958 -635530.248)   

      used_for_analysis  nwsli  plotme source  
20                 True     20    True 